# Data Extraction And Preparation

In [1]:
import numpy as np
import os,cv2,math,glob,random
from math import cos, sin
import scipy.io as sio
from pathlib import Path
import pandas as pd
import mediapipe as mp
import matplotlib.pyplot as plt

In [2]:
# store the names of the images in the folder

names=[]
for i in glob.glob(r"C:\Users\AHMED OSAMA\Desktop\project\AFLW2000/*.mat"):
    names.append(Path(i).stem)


In [3]:
# Initialize MediaPipe FaceMesh
# not normalized data !

face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True)
data=[]

for n in names:    
# for n in ["image00053","image00054"]: 
    record=[]
    image = cv2.imread(r"C:\Users\AHMED OSAMA\Desktop\project\AFLW2000\\"+n+".jpg")
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))         # it converts the image from BGR to RGB color space
    if results.multi_face_landmarks:
        for facial_landmarks in results.multi_face_landmarks:
            for i, landmark in enumerate(facial_landmarks.landmark):            # i is the index of the landmark, landmark is the pixel location
                record.append(int(landmark.x * image.shape[1]))                 # Scale the x coordinate to the width of the image
                record.append(int(landmark.y * image.shape[0]))                 # Scale the y coordinate to the height of the image
    data.append(record)

# data
data=pd.DataFrame(data)
data.columns = [f'x_{i//2 + 1}' if i % 2 == 0 else f'y_{i//2 + 1}' for i in range(len(data.columns))]


In [4]:
data

,x_1,y_1,x_2,y_2,x_3,y_3,x_4,y_4,x_5,y_5,...,x_464,y_464,x_465,y_465,x_466,y_466,x_467,y_467,x_468,y_468
0,218.0,309.0,220.0,287.0,220.0,291.0,215.0,253.0,220.0,279.0,...,251.0,218.0,246.0,221.0,243.0,225.0,299.0,212.0,304.0,208.0
1,198.0,288.0,187.0,266.0,201.0,274.0,196.0,245.0,187.0,259.0,...,235.0,228.0,230.0,230.0,225.0,231.0,263.0,225.0,267.0,222.0
2,143.0,359.0,144.0,349.0,143.0,351.0,139.0,335.0,144.0,345.0,...,150.0,323.0,148.0,324.0,147.0,325.0,168.0,320.0,169.0,319.0
3,226.0,312.0,211.0,294.0,220.0,296.0,199.0,265.0,207.0,286.0,...,228.0,225.0,223.0,229.0,220.0,233.0,268.0,205.0,271.0,200.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,221.0,311.0,225.0,286.0,223.0,292.0,220.0,251.0,226.0,277.0,...,256.0,220.0,250.0,223.0,247.0,226.0,303.0,216.0,308.0,213.0
1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Initialize MediaPipe FaceMesh
# NORMALIZE THE DATA !

face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True)
# it is recommended to make the static_image_mode True when working with pictures

data=[]

for n in names:    
    record=[]
    image = cv2.imread(r"C:\Users\AHMED OSAMA\Desktop\project\AFLW2000\\"+n+".jpg")
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))                          # it converts the image from BGR to RGB color space
    if results.multi_face_landmarks:
        for facial_landmarks in results.multi_face_landmarks:

            nose_tip = facial_landmarks.landmark[4]                                              # the nose tip landmark is at index 5
            chin = facial_landmarks.landmark[152]                                                # the chip landmark is at index 152

            norm_fact=np.sqrt((nose_tip.x-chin.x)**2+(nose_tip.y-chin.y)**2)

            for landmark in facial_landmarks.landmark:            
                record.append(((landmark.x - nose_tip.x)/(norm_fact)))                            # Scale the x coordinate 
                record.append(((landmark.y - nose_tip.y)/(norm_fact)))                            # Scale the y coordinate 
    data.append(record)                                                                           # now the data is normalized to the nose tip, and scalled with the max of the width and height of the face

# data
data=pd.DataFrame(data)
data.columns = [f'x_{i//2 + 1}' if i % 2 == 0 else f'y_{i//2 + 1}' for i in range(len(data.columns))]



In [4]:
data

,x_1,y_1,x_2,y_2,x_3,y_3,x_4,y_4,x_5,y_5,...,x_464,y_464,x_465,y_465,x_466,y_466,x_467,y_467,x_468,y_468
0,-0.023765,0.318035,-0.006607,0.089624,-0.003660,0.126111,-0.052435,-0.284172,0.0,0.0,...,0.334870,-0.658938,0.280271,-0.621766,0.246334,-0.585165,0.847953,-0.725765,0.898422,-0.761265
1,0.135390,0.342489,0.007973,0.080386,0.173977,0.176596,0.106425,-0.167740,0.0,0.0,...,0.561180,-0.368172,0.502940,-0.343096,0.446485,-0.326326,0.887145,-0.403468,0.937679,-0.429012
2,-0.034451,0.295173,-0.001236,0.081678,-0.029573,0.134064,-0.097803,-0.205510,0.0,0.0,...,0.121871,-0.470623,0.083497,-0.441704,0.067555,-0.418426,0.488717,-0.515062,0.528595,-0.545855
3,0.206938,0.278244,0.038110,0.079997,0.136178,0.103737,-0.085015,-0.232270,0.0,0.0,...,0.224828,-0.670657,0.176554,-0.623308,0.141488,-0.583820,0.656938,-0.893207,0.691559,-0.947129
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.056898,0.364123,-0.013462,0.097607,-0.029582,0.156435,-0.064382,-0.279807,0.0,0.0,...,0.324614,-0.612608,0.261693,-0.582368,0.225926,-0.553400,0.822239,-0.653323,0.873485,-0.689914
1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# store the angle of the images in the folder

angles=[]
for a in names:
    mat_file = sio.loadmat(r"C:\Users\AHMED OSAMA\Desktop\project\AFLW2000\\"+a+".mat")
    pose_para = mat_file["Pose_Para"][0][:3]
    pitch = pose_para[0]
    yaw = pose_para[1]
    roll = pose_para[2]
    angles.append([pitch,yaw,roll])

angles=pd.DataFrame(angles,columns=["pitch","yaw","roll"])



In [7]:
angles


,pitch,yaw,roll
0,-0.399231,0.018227,0.085676
1,0.470065,1.189533,0.300959
2,-0.184650,0.881137,-0.236852
3,-0.175379,0.299208,-0.373374
4,-0.882169,1.198004,-1.033374
...,...,...,...
1995,-0.197102,-0.070430,0.105118
1996,-0.232617,-1.418751,0.175960
1997,-1.447955,1.431267,-1.509418
1998,-0.420465,-1.191176,0.451515


In [8]:
angles.isna().sum()

pitch    0
yaw      0
roll     0
dtype: int64

In [9]:
data.x_1.isna().sum()

147

In [10]:
# the pic that failed to be processed
data[data.x_1.isna()]

# the index of the pic that failed to be processed
fail_i=data[data.x_1.isna()].index

# the names of the failed pics
fail_names=[]
for i in fail_i:
    fail_names.append(names[i])
    print(names[i])



image00010
image00032
image00036
image00074
image00108
image00118
image00139
image00150
image00170
image00176
image00260
image00290
image00339
image00352
image00392
image00451
image00469
image00473
image00477
image00482
image00485
image00610
image00731
image00734
image00801
image00866
image00935
image00983
image01207
image01373
image01376
image01419
image01486
image01509
image01530
image01573
image01586
image01591
image01593
image01604
image01633
image01657
image01662
image01671
image01720
image01738
image01739
image01748
image01767
image01773
image01786
image01791
image01818
image01824
image01845
image01846
image01847
image01854
image01869
image01916
image01944
image02000
image02038
image02061
image02108
image02139
image02156
image02179
image02190
image02249
image02254
image02278
image02295
image02373
image02432
image02448
image02449
image02474
image02522
image02529
image02540
image02553
image02598
image02603
image02604
image02612
image02629
image02674
image02721
image02745
image02746

In [11]:
len(fail_names)

147

In [12]:
conc_data=pd.concat([data,angles],axis=1)
conc_data

,x_1,y_1,x_2,y_2,x_3,y_3,x_4,y_4,x_5,y_5,...,y_465,x_466,y_466,x_467,y_467,x_468,y_468,pitch,yaw,roll
0,-0.023765,0.318035,-0.006607,0.089624,-0.003660,0.126111,-0.052435,-0.284172,0.0,0.0,...,-0.621766,0.246334,-0.585165,0.847953,-0.725765,0.898422,-0.761265,-0.399231,0.018227,0.085676
1,0.135390,0.342489,0.007973,0.080386,0.173977,0.176596,0.106425,-0.167740,0.0,0.0,...,-0.343096,0.446485,-0.326326,0.887145,-0.403468,0.937679,-0.429012,0.470065,1.189533,0.300959
2,-0.034451,0.295173,-0.001236,0.081678,-0.029573,0.134064,-0.097803,-0.205510,0.0,0.0,...,-0.441704,0.067555,-0.418426,0.488717,-0.515062,0.528595,-0.545855,-0.184650,0.881137,-0.236852
3,0.206938,0.278244,0.038110,0.079997,0.136178,0.103737,-0.085015,-0.232270,0.0,0.0,...,-0.623308,0.141488,-0.583820,0.656938,-0.893207,0.691559,-0.947129,-0.175379,0.299208,-0.373374
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.882169,1.198004,-1.033374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.056898,0.364123,-0.013462,0.097607,-0.029582,0.156435,-0.064382,-0.279807,0.0,0.0,...,-0.582368,0.225926,-0.553400,0.822239,-0.653323,0.873485,-0.689914,-0.197102,-0.070430,0.105118
1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.232617,-1.418751,0.175960
1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.447955,1.431267,-1.509418
1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.420465,-1.191176,0.451515


In [13]:
data[data.x_1.isna()].shape

(147, 936)

In [14]:
conc_data.dropna(inplace=True)
conc_data.reset_index(drop=True,inplace=True)
conc_data.isna().sum()

x_1      0
y_1      0
x_2      0
y_2      0
x_3      0
        ..
x_468    0
y_468    0
pitch    0
yaw      0
roll     0
Length: 939, dtype: int64

In [15]:
conc_data.iloc[:,-5:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1853 entries, 0 to 1852
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x_468   1853 non-null   float64
 1   y_468   1853 non-null   float64
 2   pitch   1853 non-null   float64
 3   yaw     1853 non-null   float64
 4   roll    1853 non-null   float64
dtypes: float64(5)
memory usage: 72.5 KB


In [16]:
conc_data

,x_1,y_1,x_2,y_2,x_3,y_3,x_4,y_4,x_5,y_5,...,y_465,x_466,y_466,x_467,y_467,x_468,y_468,pitch,yaw,roll
0,-0.023765,0.318035,-0.006607,0.089624,-0.003660,0.126111,-0.052435,-0.284172,0.0,0.0,...,-0.621766,0.246334,-0.585165,0.847953,-0.725765,0.898422,-0.761265,-0.399231,0.018227,0.085676
1,0.135390,0.342489,0.007973,0.080386,0.173977,0.176596,0.106425,-0.167740,0.0,0.0,...,-0.343096,0.446485,-0.326326,0.887145,-0.403468,0.937679,-0.429012,0.470065,1.189533,0.300959
2,-0.034451,0.295173,-0.001236,0.081678,-0.029573,0.134064,-0.097803,-0.205510,0.0,0.0,...,-0.441704,0.067555,-0.418426,0.488717,-0.515062,0.528595,-0.545855,-0.184650,0.881137,-0.236852
3,0.206938,0.278244,0.038110,0.079997,0.136178,0.103737,-0.085015,-0.232270,0.0,0.0,...,-0.623308,0.141488,-0.583820,0.656938,-0.893207,0.691559,-0.947129,-0.175379,0.299208,-0.373374
4,0.088346,0.338087,0.018155,0.080560,0.046324,0.159416,-0.083512,-0.169726,0.0,0.0,...,-0.385039,0.070019,-0.368888,0.458136,-0.529165,0.493518,-0.545693,-0.026812,0.011965,-0.220662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,-0.099509,0.382300,-0.012532,0.091444,-0.082647,0.161951,-0.118476,-0.237458,0.0,0.0,...,-0.521384,0.023654,-0.495524,0.468628,-0.638928,0.502156,-0.664404,-0.306358,-0.283822,0.038554
1849,-0.114417,0.343311,-0.014926,0.089406,-0.089372,0.143154,-0.106609,-0.248061,0.0,0.0,...,-0.538981,0.030159,-0.510099,0.424914,-0.634090,0.461930,-0.675646,-0.367547,-0.429723,0.122791
1850,-0.100844,0.302607,-0.022521,0.075017,-0.068619,0.153835,-0.028862,-0.183603,0.0,0.0,...,-0.308145,0.170728,-0.299684,0.540394,-0.272573,0.580638,-0.305275,-0.156035,0.567114,-0.108536
1851,-0.056898,0.364123,-0.013462,0.097607,-0.029582,0.156435,-0.064382,-0.279807,0.0,0.0,...,-0.582368,0.225926,-0.553400,0.822239,-0.653323,0.873485,-0.689914,-0.197102,-0.070430,0.105118


In [17]:
conc_data.to_csv(r"C:\Users\AHMED OSAMA\Desktop\project\data_centered.csv",index=False)